In [1]:
import pandas as pd
import numpy as np
import time
import datetime

%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

### Carga de datos de entrada

In [2]:
# MOVIMIENTOS

#DF = pd.read_csv('../../data/DataFrame_Final_Cierre.csv', parse_dates=['FECHA'])
df_tracks = pd.read_csv('../../data/DataFrame_Final_Cierre_Cluster.csv', parse_dates=['FECHA'])

df_clusterFull = pd.read_csv('../../data/Clusterizada_DataFull.csv')
df_clusterGeog = pd.read_csv('../../data/Clusterizada_soloGeografico.csv')
df_clusterDmnd = pd.read_csv('../../data/Clusterizada_soloDemandaRatio.csv')


In [3]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142248 entries, 0 to 2142247
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             int64         
 1   ANIO                 int64         
 2   MES                  int64         
 3   DIA                  int64         
 4   HORA                 int64         
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           int64         
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   TEMPORADA_NUM        int64         
 10  Es_Festivo           int64         
 11  Es_FinSemana         int64         
 12  TEMPERATURA          float64       
 13  VIENTO               float64       
 14  PRESION              int64         
 15  HUMEDAD              int64         
 16  PRECIPITACION_1h     float64       
 17  PRECIPITACION_3h     float64       
 18  DESC_TIEMPO          object        
 19  DESC_TIEMPO_detalle  

In [4]:
# ESTACIONES: Lista de estaciones en DF

df_estaciones = df_tracks.ESTACION.unique()
df_estaciones

array([  6,   8,   9,  12,  15,  17,  19,  20,  21,  25,  26,  31,  36,
        39,  40,  41,  43,  44,  45,  47,  48,  49,  50,  52,  53,  54,
        55,  56,  57,  58,  59,  63,  65,  66,  67,  69,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  85,  87,  90,  92,  93,
        95,  96,  98, 101, 102, 103, 104, 105, 106, 107, 110, 113, 114,
       122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 139, 145, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158,
       160, 161, 162, 163, 164, 166, 167, 168, 170, 172, 175,   7,  11,
        13,  14,  16,  18,  27,  34,  51,  64,  71,  83,  91,  94, 118,
       140, 141, 142, 143, 147,   5,  42,  46,  62,  86, 117, 121, 137,
       165,   4, 111, 116, 146, 151,  30,  89, 108, 112, 159, 169,   3,
        60,  88,  97, 109, 115, 120, 138, 171,  33,  84,  99, 127, 144,
       173, 174,   1, 100,  10,  35,  38,   2,  32,  37, 119,  61,  29,
        28, 179, 191, 178, 195, 199, 215, 192, 193, 198], dtype=

In [5]:
# Meteorologia

db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Meteo_US_NivelHora')

t_ini = time.time()
#data_Meteo = _consulta_meteoUS_por_anio(db_Meteo, 2020)
data_Meteo = _consulta_meteoUS_full(db_Meteo)
t_end = time.time()

print (t_end - t_ini)

# Se agrega campo FECHA para join
data_Meteo['FECHA'] =pd.to_datetime({'year': data_Meteo['ANIO'],
                                          'month': data_Meteo["MES"]
                                          ,'day':  data_Meteo['DIA']
                                          ,'hour':data_Meteo['HORA']},
                                          format='%d-%m-%Y', errors='coerce')

data_Meteo.info()

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_US_NivelHora')
1.0926785469055176
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ANIO                 26304 non-null  int64         
 1   MES                  26304 non-null  int64         
 2   DIA                  26304 non-null  int64         
 3   HORA                 26304 non-null  int64         
 4   TEMPERATURA          26304 non-null  object        
 5 

### Creación de DF de fechas

In [6]:
# FECHAS DE FESTIVOS

df_festivos_in = pd.read_csv('../../data/Festivos_Madrid.csv', sep=';')
# Se agrega campo FECHA para join
df_festivos_in['FECHA'] =pd.to_datetime({'year': df_festivos_in['ANIO'],
                                          'month': df_festivos_in["MES"]
                                          ,'day':  df_festivos_in['DIA']},
                                          format='%d-%m-%Y', errors='coerce')
df_festivos_in.info()
df_festivos = df_festivos_in[df_festivos_in['FESTIVO']==1]

df_festivos.info()
df_festivos[df_festivos['ANIO']==2018].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     1461 non-null   int64         
 1   MES      1461 non-null   int64         
 2   DIA      1461 non-null   int64         
 3   FESTIVO  1461 non-null   int64         
 4   FECHA    1461 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 57.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 1454
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     57 non-null     int64         
 1   MES      57 non-null     int64         
 2   DIA      57 non-null     int64         
 3   FESTIVO  57 non-null     int64         
 4   FECHA    57 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 2.7 KB


,ANIO,MES,DIA,FESTIVO,FECHA
365,2018,1,1,1,2018-01-01
370,2018,1,6,1,2018-01-06
452,2018,3,29,1,2018-03-29
453,2018,3,30,1,2018-03-30
485,2018,5,1,1,2018-05-01


### Creación de rango total de fechas

In [7]:
# Creación de rangos de fecha válidos
# sdate = datetime.strptime('01/01/2018 00', '%d/%m/%Y %H')
# edate = datetime.strptime('31/01/2018 23', '%d/%m/%Y %H')

sdate = np.datetime64('2018-01-01T00:00:00')
edate = np.datetime64('2020-02-29T23:00:00')

df_dates=pd.date_range(start=sdate, end=edate, freq='H')

print(df_dates.shape)
df_dates

(18960,)


DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00', '2018-01-01 03:00:00',
               '2018-01-01 04:00:00', '2018-01-01 05:00:00',
               '2018-01-01 06:00:00', '2018-01-01 07:00:00',
               '2018-01-01 08:00:00', '2018-01-01 09:00:00',
               ...
               '2020-02-29 14:00:00', '2020-02-29 15:00:00',
               '2020-02-29 16:00:00', '2020-02-29 17:00:00',
               '2020-02-29 18:00:00', '2020-02-29 19:00:00',
               '2020-02-29 20:00:00', '2020-02-29 21:00:00',
               '2020-02-29 22:00:00', '2020-02-29 23:00:00'],
              dtype='datetime64[ns]', length=18960, freq='H')

### DEFINICION DE FUNCIONES

In [8]:
def _estacionDataFechaNueva(idEstacion, fecha, 
                       df_clusterFull_in, df_clusterGeog_in, df_clusterDmnd_in, data_Meteo_in, df_festivos_in):
    # Variable dia semana
    if fecha.isoweekday()==7:
        dia_semana=1
    else:
        dia_semana=fecha.isoweekday()+1

    # Variable Es_FinSemana
    if ((dia_semana==1) or (dia_semana==7)):
        es_finsemana = 1
    else:
        es_finsemana = 0

    # Variable Es_Festivo
    fec_festivo = np.datetime64(fecha.date())        
    if fec_festivo in df_festivos_in['FECHA'].values:
        es_festivo = 1
    else:
        es_festivo = 0

    # Variable AM_PM
    if fecha.hour<=12:
        ampm='AM'
    else:
        ampm='PM'

    # Variable TEMPORADA
    if fecha.month<=3:
        temporada='INVIERNO'
        temporada_num=1
    elif fecha.month<=6:
        temporada='PRIMAVERA'
        temporada_num=2
    elif fecha.month<=9:
        temporada='VERANO'
        temporada_num=3
    else:
        temporada='OTONO'
        temporada_num=4

    clusterFull = df_clusterFull_in.loc[df_clusterFull_in['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']        
    clusterGeog = df_clusterGeog_in.loc[df_clusterGeog_in['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']
    clusterDmnd = df_clusterDmnd_in.loc[df_clusterDmnd_in['ESTACION'] == estacion, ['CLUSTER']].iloc[0]['CLUSTER']
    
    meteo = data_Meteo_in[data_Meteo_in['FECHA']==fecha]
    
    new_row_dict = {
        'FECHA': fecha,
        'ESTACION': idEstacion,
        'ANIO': fecha.year,
        'MES': fecha.month,
        'DIA': fecha.day,
        'HORA': fecha.hour,
        'DIA_SEMANA': dia_semana,
        'AM_PM': ampm,
        'TEMPORADA': temporada,
        'TEMPORADA_NUM': temporada_num,
        'Es_Festivo': es_festivo,
        'Es_FinSemana': es_finsemana,
        'TEMPERATURA': float(meteo.TEMPERATURA.iloc[0]),
        'VIENTO': float(meteo.VIENTO.iloc[0]),
        'PRESION': float(meteo.PRESION.iloc[0]),
        'HUMEDAD': float(meteo.HUMEDAD.iloc[0]),
        'PRECIPITACION_1h': float(meteo.PRECIPITACION_1h.iloc[0]),
        'PRECIPITACION_3h': float(meteo.PRECIPITACION_3h.iloc[0]),
        'DESC_TIEMPO': meteo.DESC_TIEMPO.iloc[0],
        'DESC_TIEMPO_detalle': meteo.DESC_TIEMPO_detalle.iloc[0],
        'CLUSTER_FullCols': clusterFull,
        'CLUSTER_soloGeo': clusterGeog,
        'CLUSTER_soloDemanda': clusterDmnd,
        'DEMANDA': 0
    }

    return pd.DataFrame(new_row_dict, index=[0])
    

### EJECUCION DE PROCESO v1

In [ ]:
df_estaciones = [175]
type(df_estaciones)

In [ ]:
i=0
df_final = pd.DataFrame(columns = ['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 
                            'TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 
                            'PRECIPITACION_1h', 'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'CLUSTER_FullCols', 
                            'CLUSTER_soloGeo', 'CLUSTER_soloDemanda', 'DEMANDA'])

t_ini = time.time()

for estacion in df_estaciones:
    print('Estacion: '+str(estacion))
    
    df_data = df_tracks[df_tracks['ESTACION']==estacion]
    
    df_final_estacion = pd.DataFrame(columns = ['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 
                            'TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 
                            'PRECIPITACION_1h', 'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'CLUSTER_FullCols', 
                            'CLUSTER_soloGeo', 'CLUSTER_soloDemanda', 'DEMANDA'])    
    
    t_ini_estacion = time.time()
    for fecha in df_dates:
        
        if fecha not in df_data[df_data['ESTACION']==estacion]['FECHA'].values:
        # fecha NO existe en BD de movimientos para estacion
            i=i+1
                      
            new_row = _estacionDataFechaNueva(estacion, fecha, df_clusterFull, df_clusterGeog, df_clusterDmnd, 
                                              data_Meteo, df_festivos)
            
            df_final_estacion = pd.concat([df_final_estacion, new_row])
            
        else:
        # fecha SI existe en BD de movimientos para estacion
            old_row = df_data[(df_data['ESTACION']==estacion) & (df_data['FECHA']==fecha)]
            df_final_estacion = pd.concat([df_final_estacion, old_row])
            
            
    df_final = pd.concat([df_final, df_final_estacion])
    
    t_end_estacion = time.time()
    print('Fechas vacías:' + str(i))
    print('Tiempo: '+str(t_end_estacion-t_ini_estacion))
    i=0
    

t_end = time.time()
print (t_end - t_ini)

### Ejecución Proceso v2

In [10]:
df_dates_DF = pd.DataFrame(df_dates)
df_dates_DF = df_dates_DF.rename(columns={0: 'FECHA'})
df_dates_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18960 entries, 0 to 18959
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FECHA   18960 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 148.2 KB


In [9]:
df_estaciones = [175]
df_estaciones

[175]

In [ ]:
i=0
df_final = pd.DataFrame(columns = ['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 
                            'TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 
                            'PRECIPITACION_1h', 'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'CLUSTER_FullCols', 
                            'CLUSTER_soloGeo', 'CLUSTER_soloDemanda', 'DEMANDA'])

t_ini = time.time()

for estacion in df_estaciones:
    # Data de estacion
    df_tracks_estacion = df_tracks[df_tracks['ESTACION']==estacion]
    
    # Merge entre fechas y data de estacion
    df_tracks_estacion = pd.merge(df_dates_DF, df_tracks_estacion, how='left', left_on='FECHA', right_on='FECHA')
    
    # Búsqueda de registros con valores nulos
    #df_tracks_estacion[df_tracks_estacion['ANIO'].isna()]

#     df_tracks_estacion['ESTACION'].fillna(estacion, inplace=True)    
#     df_tracks_estacion['DEMANDA'].fillna(0, inplace=True)

    df_tracks_local = df_tracks_estacion.copy()
    
    for index, row in df_tracks_estacion.iterrows():
        print(df_tracks_local.loc[index]['FECHA'])
        
        if pd.isnull(row.loc['ANIO']):
            new_row = _estacionDataFechaNueva(estacion, row.loc['FECHA'], df_clusterFull, df_clusterGeog, df_clusterDmnd, 
                                              data_Meteo, df_festivos)

            #print('indice: ' + str(index) + '; FECHA: ' + str(new_row['FECHA'])+ '; DEMANDA: ' + str(new_row['DEMANDA']))
            #df_tracks_local[index:] = new_row
            print(df_tracks_local.loc[index].T)
#             df_final = pd.concat([df_final, new_row])
#         else:
#             df_final = pd.concat([df_final, row.T])

    df_final = pd.concat([df_final, df_tracks_local])

In [30]:
df_final[['ESTACION', 'FECHA', 'DEMANDA']][df_final['ESTACION']==175]

,ESTACION,FECHA,DEMANDA
0,175.0,2018-01-01 00:00:00,5.0
1,175.0,2018-01-01 01:00:00,0.0
2,175.0,2018-01-01 01:00:00,0.0
3,175.0,2018-01-01 01:00:00,0.0
4,175.0,2018-01-01 04:00:00,0.0
...,...,...,...
18955,175.0,2020-02-29 05:00:00,0.0
18956,175.0,2020-02-29 05:00:00,0.0
18957,175.0,2020-02-29 05:00:00,0.0
18958,175.0,2020-02-29 05:00:00,0.0


In [27]:
df_tracks[['ESTACION', 'FECHA', 'DEMANDA']][df_tracks['ESTACION']==175]

,ESTACION,FECHA,DEMANDA
101,175,2018-01-01 00:00:00,5
244,175,2018-01-01 02:00:00,1
302,175,2018-01-01 03:00:00,1
466,175,2018-01-01 05:00:00,6
540,175,2018-01-01 06:00:00,1
...,...,...,...
2141766,175,2020-02-29 19:00:00,8
2141900,175,2020-02-29 20:00:00,3
2142019,175,2020-02-29 21:00:00,1
2142118,175,2020-02-29 22:00:00,2


In [28]:
df_tracks_local[['ESTACION', 'FECHA', 'DEMANDA']][df_tracks['ESTACION']==175]

C:\Users\gonza\AppData\Local\Temp\ipykernel_2204\1180602783.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_tracks_local[['ESTACION', 'FECHA', 'DEMANDA']][df_tracks['ESTACION']==175]


,ESTACION,FECHA,DEMANDA
101,175.0,2020-02-29 05:00:00,0.0
244,175.0,2020-02-29 05:00:00,0.0
302,175.0,2020-02-29 05:00:00,0.0
466,175.0,2020-02-29 05:00:00,0.0
540,175.0,2020-02-29 05:00:00,0.0
...,...,...,...
18424,175.0,2020-02-29 05:00:00,0.0
18574,175.0,2020-02-29 05:00:00,0.0
18704,175.0,2020-02-29 05:00:00,0.0
18827,175.0,2020-02-29 05:00:00,0.0


In [ ]:
df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA','sum'))

In [ ]:
df_tracks[df_tracks['ESTACION']==175].groupby('ESTACION').agg(DEMANDA=('DEMANDA','sum'))

In [ ]:
df_tracks[df_tracks['ESTACION']==175]

In [ ]:
df_tracks_estacion = df_tracks[df_tracks['ESTACION']==175]

df_tracks_estacion = pd.merge(df_dates_DF, df_tracks_estacion, how='left', left_on='FECHA', right_on='FECHA')

df_tracks_estacion

In [ ]:
df_final.info()

In [ ]:
df_final['CLUSTER_FullCols'] = df_final['CLUSTER_FullCols'].astype(np.int64)
df_final['CLUSTER_soloGeo'] = df_final['CLUSTER_soloGeo'].astype(np.int64)
df_final['CLUSTER_soloDemanda'] = df_final['CLUSTER_soloDemanda'].astype(np.int64)
df_final['TEMPORADA_NUM'] = df_final['TEMPORADA_NUM'].astype(np.int64)
df_final['DEMANDA'] = df_final['DEMANDA'].astype(np.int64)

df_final.info()

In [ ]:
df_final

In [ ]:
df_final[df_final['DEMANDA']==0]

In [ ]:
df_final.groupby('CLUSTER_soloGeo').count()

### Validación de resultados en base a suma de DEMANDA

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
# DF Original
df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

In [ ]:
# DF Corregido
df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

In [ ]:
df_final.info()

In [ ]:
pd.set_option('display.max_rows', 10)

### Export df_final a CSV

In [ ]:
df_final.to_csv('../../data/DataFrame_Final_Cierre_Cluster_FullFecha.csv', index=False)
#df_final.to_csv('../../data/test.csv', index=False)

In [ ]:
test=df_final.groupby(by='TEMPORADA').count()
test

In [ ]:
test=df_final.groupby(by='ANIO').count()
test

In [ ]:
df_final.groupby('ANIO').agg(DEMANDA=('DEMANDA', 'sum'))

In [ ]:
DF2 = pd.read_csv('../../data/test.csv', parse_dates=['FECHA'])
DF2.info()